In [1]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
# %cd /content/drive/MyDrive/CDCN/CDCN-Face-Anti-Spoofing.pytorch

/content/drive/MyDrive/CDCN/CDCN-Face-Anti-Spoofing.pytorch


In [3]:
import os
import torch
from torchvision import transforms, models
from torch.utils.tensorboard import SummaryWriter
from datasets.FASDataset import FASDataset
from utils.transform import RandomGammaCorrection
from utils.utils import read_cfg, get_optimizer, get_device, build_network
from trainer.FASTrainer import FASTrainer
from models.loss import DepthLoss
from torch.optim.lr_scheduler import StepLR

In [4]:
cfg = read_cfg(cfg_file="config/CDCNpp_adam_lr1e-3.yaml")
cfg["device"]="0"
cfg["dataset"]["name"] = "my_data"
cfg["dataset"]["root"] = "data/my_data"
cfg["train"]["num_epochs"] = 20

device = get_device(cfg)

network = build_network(cfg)

optimizer = get_optimizer(cfg, network)

In [5]:
device

device(type='cuda', index=0)

In [6]:
cfg

{'output_dir': 'experiments/output',
 'log_dir': 'experiments/log',
 'device': '0',
 'dataset': {'name': 'my_data',
  'root': 'data/my_data',
  'augmentation': {'horizontal_flip': True,
   'rotation_range': 10,
   'gamma_correction': ['1.0/2.2', '2.2/1.0'],
   'brightness': 0.5,
   'contrast': 0.5,
   'saturation': 0.5,
   'hue': 0.5},
  'train_set': 'train.csv',
  'val_set': 'val.csv',
  'mean': [0.5, 0.5, 0.5],
  'sigma': [0.5, 0.5, 0.5]},
 'model': {'base': 'CDCN',
  'pretrained': False,
  'input_size': [256, 256],
  'depth_map_size': [32, 32]},
 'train': {'batch_size': 24,
  'optimizer': 'adam',
  'lr': 0.001,
  'num_epochs': 20,
  'smoothing': True},
 'val': {'batch_size': 24}}

In [7]:
lr_scheduler = StepLR(optimizer=optimizer, step_size=30, gamma=0.1)

criterion = DepthLoss(device=device)

writer = SummaryWriter(cfg['log_dir'])

dump_input = torch.randn((1, 3, cfg['model']['input_size'][0], cfg['model']['input_size'][1]))

writer.add_graph(network, dump_input)

In [8]:
train_transform = transforms.Compose([
    RandomGammaCorrection(max_gamma=cfg['dataset']['augmentation']['gamma_correction'][1],
                            min_gamma=cfg['dataset']['augmentation']['gamma_correction'][0]),
    transforms.RandomResizedCrop(cfg['model']['input_size'][0]),

    transforms.RandomRotation(cfg['dataset']['augmentation']['rotation_range']),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(cfg['model']['input_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])
])

In [9]:
val_transform = transforms.Compose([
    transforms.Resize(cfg['model']['input_size']),
    transforms.ToTensor(),
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma'])
])

In [10]:
trainset = FASDataset(
    root_dir=cfg['dataset']['root'],
    csv_file=cfg['dataset']['train_set'],
    depth_map_size=cfg['model']['depth_map_size'],
    transform=train_transform,
    smoothing=cfg['train']['smoothing']
)

In [11]:
valset = FASDataset(
    root_dir=cfg['dataset']['root'],
    csv_file=cfg['dataset']['val_set'],
    depth_map_size=cfg['model']['depth_map_size'],
    transform=val_transform,
    smoothing=cfg['train']['smoothing']
)

In [12]:
trainloader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=cfg['train']['batch_size'],
    shuffle=True,
    num_workers=2
)

In [13]:
valloader = torch.utils.data.DataLoader(
    dataset=valset,
    batch_size=cfg['val']['batch_size'],
    shuffle=True,
    num_workers=2
)

In [14]:
import torch

# Check if CUDA (GPU) is available
print(torch.cuda.is_available())


True


In [15]:
trainer = FASTrainer(
    cfg=cfg,
    network=network,
    optimizer=optimizer,
    criterion=criterion,
    lr_scheduler=lr_scheduler,
    device=device,
    trainloader=trainloader,
    valloader=valloader,
    writer=writer
)

In [16]:
network

CDCN(
  (conv1): Sequential(
    (0): Conv2d_cd(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (Block1): Sequential(
    (0): Conv2d_cd(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d_cd(
      (conv): Conv2d(128, 196, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (4): BatchNorm2d(196, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d_cd(
      (conv): Conv2d(196, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): MaxPool2d(kernel_size=3, stride=2, 

In [17]:
history = trainer.train()

Streaming output truncated to the last 5000 lines.
Epoch: 0, iter: 100, loss: 0.3326095437059308, acc: 0.5688943904225189
Epoch: 0, iter: 101, loss: 0.33211126280765907, acc: 0.5686274521491107
Epoch: 0, iter: 102, loss: 0.33149897878609813, acc: 0.5687702275016933
Epoch: 0, iter: 103, loss: 0.3309849234154591, acc: 0.5685096165308585
Epoch: 0, iter: 104, loss: 0.3302662894839332, acc: 0.5690476201829456
Epoch: 0, iter: 105, loss: 0.32970678750074134, acc: 0.5691823908742869
Epoch: 0, iter: 106, loss: 0.329265654365593, acc: 0.5689252347589653
Epoch: 0, iter: 107, loss: 0.32859324001603657, acc: 0.5694444455482341
Epoch: 0, iter: 108, loss: 0.3280150081039569, acc: 0.569571866354811
Epoch: 0, iter: 109, loss: 0.32732542861591685, acc: 0.5700757584788583
Epoch: 0, iter: 110, loss: 0.32752313753506085, acc: 0.5675675683730358
Epoch: 0, iter: 111, loss: 0.3270284679851362, acc: 0.567708333954215
Epoch: 0, iter: 112, loss: 0.3269195844114354, acc: 0.5660029504678946
Epoch: 0, iter: 113, lo

In [ ]:
import torch

# Load the model architecture
from models.CDCNs import CDCNpp  # Replace with your model class import

# Instantiate the model
model = CDCNpp()

# Load the trained weights
checkpoint = torch.load('experiments/output/CDCNpp_nuaa.pth',map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['state_dict'])
model.eval()  # Set the model to evaluation mode

from ultralytics import YOLO
import cv2
import tensorflow as tf 
import keras
from keras.models import Sequential
from tensorflow.keras.models import load_model
import numpy as np
import os

preprocess = transforms.Compose([
    transforms.ToPILImage(),  # Convert to PIL Image
    transforms.Resize(cfg['model']['input_size']),  # Resize
    transforms.ToTensor(),  # Convert to tensor
    transforms.Normalize(cfg['dataset']['mean'], cfg['dataset']['sigma']),  # Normalize
])

face_model = YOLO("models/yolov8n-face.pt")

# Open the camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    

    # Display the live video stream
    cv2.imshow("Camera", frame)

    # Capture an image when 'c' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('c'):
        
        results = face_model(frame)
        
        input_tensor = preprocess(frame).unsqueeze(0)
        with torch.no_grad():
            outputs = model(input_tensor)
        
        
        predicted_classes = torch.argmax(outputs[0], dim=1)
        
        yhat = torch.mode(predicted_classes).values.item()
        
        if yhat == 1:
            label = 'Real'
        else:
            label = 'Fake'


        for result in results[0].boxes:
            top_left_x = int(result.xyxy.tolist()[0][0])
            top_left_y = int(result.xyxy.tolist()[0][1])
            bottom_right_x = int(result.xyxy.tolist()[0][2])
            bottom_right_y = int(result.xyxy.tolist()[0][3])
        
        
        cv2.rectangle(frame, (top_left_x,top_left_y), (bottom_right_x,bottom_right_y), (255, 0, 0), 2)
        cv2.putText(frame, label, (top_left_x, top_left_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
        
        cv2.imshow("Captured Image", frame)

        
        cv2.waitKey(0)
        cv2.destroyAllWindows()

    # Press 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()
